**a. Import required libraries**

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

**b. Loading dataset**

assuming data is your dataset

In [26]:
Path_TO_DATA = 'https://storage.googleapis.com/download.tensorflow.org/data/ecg.csv'
data = pd.read_csv(Path_TO_DATA, header = None)

split data into test and train set

In [27]:
x_train, x_test = train_test_split(data, test_size = 0.2, random_state=42)

**c. Encoder**

In [28]:
input_img = x_train.shape[1]

In [29]:
encoder = tf.keras.Sequential([
    Dense(64, activation = "relu", input_shape = (input_img,)),
    Dense(32, activation = "relu"),
    Dense(16, activation = "relu")
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**d. Decoder**

In [30]:
decoder = tf.keras.Sequential([
    Dense(32, activation = "relu", input_shape=(16,)),
    Dense(64, activation = "relu"),
    Dense(input_img, activation = "sigmoid")
])

**e. Compile and fit model**

In [31]:
# combine encoder and decoder into autoencoder model
autoencoder_input = Input(shape=(input_img,))
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)
autoencoder = Model(autoencoder_input, decoded)



In [32]:
#compile model
autoencoder.compile(
    optimizer = "adam",
    loss = "mse"
)

autoencoder.fit( x_train, x_train, epochs = 20, batch_size = 32, shuffle = True, validation_data = (x_test, x_test))


Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.0120 - val_loss: 0.7796
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7739 - val_loss: 0.7550
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7520 - val_loss: 0.7426
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7403 - val_loss: 0.7376
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7383 - val_loss: 0.7358
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7351 - val_loss: 0.7349
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7347 - val_loss: 0.7334
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7333 - val_loss: 0.7325
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7323 - val_loss: 0.7316
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7307 - val_loss: 0.7311
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7311 - val_loss: 0.7308
Epoch 12/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [33]:
# reconstruction loss
reconstruction_loss = autoencoder.evaluate(x_test, x_test)
print("Reconstruction Loss:", reconstruction_loss)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7258 
Reconstruction Loss: 0.7278206944465637


In [34]:
# anomaly detection
reconstructions = autoencoder.predict(x_test)
mse = np.mean(np.power(x_test - reconstructions, 2), axis=1)  # calculate mse per sample
threshold = np.mean(mse) + 3 * np.std(mse)  # threshold for reconstruction error


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [35]:
anomalies = np.where(mse > threshold)[0]
print("Indices of Anomalies:", anomalies)

Indices of Anomalies: [360 421 494 727]
